In [1]:
import pandas as pd
import nltk
import numpy as np

#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# df_eda=pd.read_csv('eda_np.txt', sep="\t", header=0)

In [3]:
# df_eda

In [4]:
df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    # return " ".join(new_tokens), len(new_tokens)
    return " ".join(new_tokens)

In [6]:
# df['tags_pre'] = df['tags'].apply(lambda x: x.replace(',', ' '))

In [7]:
titles = df['title_orig'].tolist()
# results = [process_sentence(t) for t in titles]
tokens = [process_sentence(t) for t in titles]
# tokens, length = zip(*results)
df['title_pre'] = tokens
# df['title_len'] = df['title_pre'].str.split(" ").str.len()

In [8]:
# df[['title_len', 'units_sold']].sort_values(by='units_sold', ascending=False).head(10).mean(axis=0)

In [9]:
# df[['title_len', 'units_sold']].sort_values(by='units_sold', ascending=False).tail(10).mean(axis=0)

In [10]:
# tags = df['tags_pre'].tolist()
# results = [process_sentence(t) for t in tags]
# tokens, length = zip(*results)
# df['tags_pre'] = tokens
# df['tags_num'] = length

In [11]:
# df[['tags_num', 'units_sold']].sort_values(by='units_sold', ascending=False).head(30).mean(axis=0)

In [12]:
# df[['tags_num', 'units_sold']].sort_values(by='units_sold', ascending=False).tail(30).mean(axis=0)

In [13]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['title_pre'])

In [14]:
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
tfidf = pd.DataFrame(denselist, columns=feature_names)

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
tfidf

,absorbing,accessory,active,activewear,activity,acute,adhesive,adjustable,aeeival,african,...,yoga,youth,zanzea,zapatillas,zapatos,zapper,ziper,zipper,zm,übergröße
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.260621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(df['tags_pre'])
# feature_names = vectorizer.get_feature_names()
# dense = vectors.todense()
# denselist = dense.tolist()
# tfidf_tag = pd.DataFrame(denselist, columns=feature_names)

In [17]:
# product color
def main_color(s):
    main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange", 
    "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white", 
    "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
    for key, value in main_color.items():
        if key in s:
            return value
    return "others"
product_color = df["product_color"]
product_color = [s.lower() if type(s) is str else 'nan' for s in product_color]
product_color = [main_color(s) for s in product_color]
from matplotlib import colors
product_color = [(-0.1,-0.1,-0.1,-0.1) if s == "others" else colors.to_rgba(s) for s in product_color]

df['product_color_rgb'] = [np.array(t) for t in product_color]

# log prices
df['log_price'] = [np.log(p) for p in df["price"]]
df['log_retail_price'] = [np.log(p) for p in df["retail_price"]]

# log merchant rating count
df['log_merchant_rating_count'] = np.log(df['merchant_rating_count'])

# urgent text
df['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df["urgency_text"]]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
data = df[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product", 
           "badge_product_quality", "badge_fast_shipping", "urgent", "units_sold"]]
df2 = pd.concat([data, tfidf], axis=1)
label = [1 if sales > 100 else 0 for sales in data["units_sold"]]
df2['high_sale'] = label
print(df2['high_sale'].value_counts())

1    925
0    648
Name: high_sale, dtype: int64


In [19]:
rgb = df["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(4):
    df2["product_color_rgb"+str(i)] = rgb[:,i]
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
print(df2.shape)

(1573, 1185)


In [20]:
X = df2.loc[:, ~df2.columns.isin(['high_sale', 'units_sold'])]
# X = tfidf
y = df2['high_sale']

In [21]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)

Using TensorFlow backend.


In [22]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1179
number of dev examples = 197
number of test examples = 197
X_train shape: (1179, 1183)
Y_train shape: (1179,)
X_dev shape: (197, 1183)
Y_dev shape: (197,)
X_test shape: (197, 1183)
Y_test shape: (197,)


In [23]:
X_train = tf.expand_dims(X_train, axis=-1)
X_dev = tf.expand_dims(X_dev, axis=-1)
input_shape = X_train.shape[1:]

In [24]:
print(input_shape)

(1183, 1)


In [25]:
# from sklearn.linear_model import LogisticRegression

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# score = classifier.score(evals_X, evals_y)

# print("Accuracy:", score)

In [26]:
from keras.layers import AveragePooling1D, PReLU, LeakyReLU

In [27]:
# define model
model = Sequential()
model.add(Conv1D(filters=24, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=30)

In [29]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    verbose=2,
    validation_data=(X_dev, y_dev),
    batch_size=16,
    callbacks=[es]
)
model.summary()

Epoch 1/1000

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

74/74 - 1s - loss: 0.9045 - accuracy: 0.4343 - val_loss: 0.6610 - val_accuracy: 0.3655
Epoch 2/1000
74/74 - 1s - loss: 0.6854 - accuracy: 0.4385 - val_loss: 0.6635 - val_accuracy: 0.4213
Epoch 3/1000
74/74 - 1s - loss: 0.6848 - accuracy: 0.4207 - val_loss: 0.6623 - val_accuracy: 0.3655
Epoch 4/1000
74/74 - 1s - loss: 0.6852 - accuracy: 0.4173 - val_loss: 0.6697 - val_accuracy: 0.3655
Epoch 5/1000
74/74 - 1s - loss: 0.6859 - accuracy: 0.4173 - val_loss: 0.6955 - val_accuracy: 0.3655
Epoch 6/

Epoch 76/1000
74/74 - 2s - loss: 0.6347 - accuracy: 0.7727 - val_loss: 0.6505 - val_accuracy: 0.6853
Epoch 77/1000
74/74 - 2s - loss: 0.6340 - accuracy: 0.7676 - val_loss: 0.6500 - val_accuracy: 0.6853
Epoch 78/1000
74/74 - 2s - loss: 0.6317 - accuracy: 0.7820 - val_loss: 0.6455 - val_accuracy: 0.6954
Epoch 79/1000
74/74 - 2s - loss: 0.6326 - accuracy: 0.7786 - val_loss: 0.6500 - val_accuracy: 0.6650
Epoch 80/1000
74/74 - 2s - loss: 0.6299 - accuracy: 0.7803 - val_loss: 0.6493 - val_accuracy: 0.6751
Epoch 81/1000
74/74 - 2s - loss: 0.6280 - accuracy: 0.7897 - val_loss: 0.6504 - val_accuracy: 0.6701
Epoch 82/1000
74/74 - 2s - loss: 0.6241 - accuracy: 0.7956 - val_loss: 0.6594 - val_accuracy: 0.5939
Epoch 83/1000
74/74 - 2s - loss: 0.6268 - accuracy: 0.7998 - val_loss: 0.6463 - val_accuracy: 0.6802
Epoch 84/1000
74/74 - 2s - loss: 0.6236 - accuracy: 0.8024 - val_loss: 0.6397 - val_accuracy: 0.6802
Epoch 85/1000
74/74 - 2s - loss: 0.6224 - accuracy: 0.8015 - val_loss: 0.6468 - val_accurac

Epoch 157/1000
74/74 - 2s - loss: 0.5768 - accuracy: 0.8821 - val_loss: 0.6347 - val_accuracy: 0.7056
Epoch 158/1000
74/74 - 2s - loss: 0.5760 - accuracy: 0.8863 - val_loss: 0.6502 - val_accuracy: 0.6294
Epoch 159/1000
74/74 - 2s - loss: 0.5762 - accuracy: 0.8796 - val_loss: 0.6432 - val_accuracy: 0.6904
Epoch 160/1000
74/74 - 2s - loss: 0.5744 - accuracy: 0.8889 - val_loss: 0.6367 - val_accuracy: 0.7056
Epoch 161/1000
74/74 - 2s - loss: 0.5697 - accuracy: 0.8991 - val_loss: 0.6479 - val_accuracy: 0.6802
Epoch 162/1000
74/74 - 2s - loss: 0.5805 - accuracy: 0.8770 - val_loss: 0.6292 - val_accuracy: 0.7056
Epoch 163/1000
74/74 - 2s - loss: 0.5831 - accuracy: 0.8694 - val_loss: 0.6447 - val_accuracy: 0.6853
Epoch 164/1000
74/74 - 2s - loss: 0.5753 - accuracy: 0.8838 - val_loss: 0.6379 - val_accuracy: 0.7056
Epoch 165/1000
74/74 - 2s - loss: 0.5740 - accuracy: 0.8863 - val_loss: 0.6324 - val_accuracy: 0.7157
Epoch 166/1000
74/74 - 2s - loss: 0.5710 - accuracy: 0.8863 - val_loss: 0.6380 - v

Epoch 238/1000
74/74 - 2s - loss: 0.5637 - accuracy: 0.8974 - val_loss: 0.6421 - val_accuracy: 0.6701
Epoch 239/1000
74/74 - 2s - loss: 0.5653 - accuracy: 0.8957 - val_loss: 0.6392 - val_accuracy: 0.6599
Epoch 240/1000
74/74 - 2s - loss: 0.5606 - accuracy: 0.9092 - val_loss: 0.6543 - val_accuracy: 0.6244
Epoch 241/1000
74/74 - 2s - loss: 0.5637 - accuracy: 0.9025 - val_loss: 0.6404 - val_accuracy: 0.6802
Epoch 242/1000
74/74 - 2s - loss: 0.5583 - accuracy: 0.9109 - val_loss: 0.6367 - val_accuracy: 0.6802
Epoch 243/1000
74/74 - 2s - loss: 0.5581 - accuracy: 0.9084 - val_loss: 0.6307 - val_accuracy: 0.6954
Epoch 244/1000
74/74 - 2s - loss: 0.5577 - accuracy: 0.9059 - val_loss: 0.6364 - val_accuracy: 0.6954
Epoch 245/1000
74/74 - 2s - loss: 0.5613 - accuracy: 0.8965 - val_loss: 0.6407 - val_accuracy: 0.6853
Epoch 246/1000
74/74 - 2s - loss: 0.5629 - accuracy: 0.8940 - val_loss: 0.6452 - val_accuracy: 0.6447
Epoch 247/1000
74/74 - 2s - loss: 0.5571 - accuracy: 0.9152 - val_loss: 0.6450 - v